## Лекция 2  BM5    

### TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1
 
# чтобы получить сгенерированный словарь, из приведенной структуры TfidfVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('слово3') # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
new_doc = vectorizer.transform(['слово1 слово4 слово4']).toarray()  # результат [[0.36673901, 0, 0, 0.93032387]]
 

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [ ]:
### реализуйте эту функцию ранжирования 
from math import log

k = 2.0
b = 0.75


def bm25() -> float:
    return 

### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 


In [1]:
import pandas as pd

answers = pd.read_excel('answers_base.xlsx')
queries = pd.read_excel("queries_base.xlsx")

можно сделать препроцессинг, потому что в примере лежат слова по попрядку в документах без знаков препинания

In [2]:
import string
import re
import pandas as pd
from pymorphy2 import MorphAnalyzer


morph = MorphAnalyzer()

In [3]:
def preprocessing(data):
    data2 = []
    cnt = 0
    for v in data:
        strng = ''
        line = "".join(l for l in v if l not in (
            '.', ',', '?', '!', ':', ';', '—', '--', '<', '>', '"', '(', ')'))
        for word in line.split():
            word = re.sub(r'\ufeff', '', word)
            wrd = morph.parse(word)[0]
            strng += wrd.normal_form + ' '
        data2.append(strng)
    return data2

Можно улучшить: апгрейдить токенизаию и добавить еще удаление стоп-слов (последнее - как в первой домашке)

In [4]:
lemm_text = preprocessing(answers['Текст вопросов'])

In [5]:
answers['вопросы_препроцессинговые'] = lemm_text

In [6]:
answers.head()

,Номер связки,Текст вопросов,Текст ответа,Тематика,вопросы_препроцессинговые
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,у ребёнок в школа продлить каникулы мочь ли я ...
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,где сделать вакцинация от коронавирус существо...
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,сколько стоить сделать вакцина от грипп можно ...
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ,мочь я отказаться от вакцинация в какой случай...
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ,безопасный ли вакцинация опасный ли вакцинация...


In [7]:
queries.head()

,Текст вопроса,Номер связки,Тематика,Unnamed: 3,Unnamed: 4
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN


In [8]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 350

train, test = train_test_split(queries, test_size=0.3,
                               random_state=RANDOM_STATE)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

In [10]:
X = vectorizer.fit_transform(answers['вопросы_препроцессинговые'])

In [11]:
X

<43x1020 sparse matrix of type '<class 'numpy.float64'>'
	with 2211 stored elements in Compressed Sparse Row format>

In [12]:
matr_answers = X.toarray()

In [13]:
matr_answers

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.25150849]])

In [14]:
import numpy as np

In [15]:
def preprocess_text(data):
    strng = ''
    line = re.sub(r'[\.\,?!:;—<>\"\'()@#$\*+=]', '', str(data))
    for word in line.split():
        wrd = morph.parse(word)[0]
        strng += wrd.normal_form + ' '
    return strng

In [16]:
def text_to_vec(text, vectorizer, matr):
    txt = preprocess_text(text)
    new_doc = vectorizer.transform([txt]).toarray()
    new_matr = matr.dot(new_doc[0])
    return new_matr

In [17]:
text = text_to_vec(queries.head(1)['Текст вопроса'].tolist()[0], vectorizer, matr_answers)

In [18]:
text

array([0.05680405, 0.02704388, 0.        , 0.03340724, 0.        ,
       0.        , 0.01403387, 0.02523288, 0.05963104, 0.05042647,
       0.08427398, 0.16265468, 0.02373525, 0.        , 0.13627539,
       0.        , 0.06425065, 0.        , 0.        , 0.05491306,
       0.03282078, 0.0445077 , 0.13916294, 0.05218801, 0.        ,
       0.17281213, 0.03895571, 0.0578759 , 0.06245531, 0.        ,
       0.01273891, 0.11696884, 0.07764165, 0.        , 0.01233948,
       0.09221815, 0.01612744, 0.12104236, 0.14894832, 0.10201655,
       0.05624647, 0.02084776, 0.10202387])

In [19]:
n_conn = answers['Номер связки'].tolist()

In [20]:
import operator

In [21]:
def ranging(text):
    
    dict_texts = {}

    cnt = 0
    for x in np.nditer(text):
        dict_texts[n_conn[cnt]] = float(x)
        cnt += 1

    new_dict = sorted(dict_texts.items(),
                      key=operator.itemgetter(1), reverse=True)
    first = []
    print('Ранжированные документы по близости: \n---\n')
    for k in new_dict:
        first.append(k[0])
        print (k[1], ': номер связки - ', k[0])

    strng = '\n---\nДля этого текста связка самого близкого ответа: ' + str(first[0])
    print(strng)

In [22]:
ranging(text)

Ранжированные документы по близости: 
---

0.1728121269222734 : номер связки -  37
0.162654675361505 : номер связки -  308
0.14894831866292457 : номер связки -  135
0.1391629405823113 : номер связки -  286
0.13627538990895594 : номер связки -  1
0.12104235745025134 : номер связки -  6
0.11696883850599675 : номер связки -  270
0.10202387046241142 : номер связки -  21
0.10201655323303854 : номер связки -  5
0.09221814775917288 : номер связки -  88
0.08427398246623416 : номер связки -  173
0.0776416527441024 : номер связки -  246
0.06425065305889403 : номер связки -  10
0.062455313519881116 : номер связки -  132
0.059631037054974405 : номер связки -  82
0.057875897699810046 : номер связки -  74
0.05680404583178923 : номер связки -  57
0.05624647341850496 : номер связки -  3
0.054913057036602105 : номер связки -  34
0.052188012235538005 : номер связки -  79
0.050426469015820624 : номер связки -  89
0.044507696321645376 : номер связки -  325
0.03895570998382648 : номер связки -  32
0.033407

In [23]:
def ranging_simple(text):
    
    dict_texts = {}

    cnt = 0
    for x in np.nditer(text):
        dict_texts[n_conn[cnt]] = float(x)
        cnt += 1

    new_dict = sorted(dict_texts.items(),
                      key=operator.itemgetter(1), reverse=True)
    first = []
    for k in new_dict:
        first.append(k[0])

    return first[0]

In [24]:
ranging_simple(text)

37

In [26]:
for_tf = []
for index, row in queries.iterrows():
    fin = text_to_vec(row['Текст вопроса'], vectorizer, matr_answers)
    for_tf.append(ranging_simple(fin))

In [27]:
queries['tf_vector'] = for_tf

In [29]:
queries.head()

,Текст вопроса,Номер связки,Тематика,Unnamed: 3,Unnamed: 4,tf_vector
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN,37
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN,324
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN,1
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN,32
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN,286


Но данные без препроцессинга иногда могут быть ближе к реальности

In [30]:
vectorizer_2 = TfidfVectorizer()

In [31]:
X_start = vectorizer_2.fit_transform(answers['Текст вопросов'])

In [32]:
X_start

<43x1554 sparse matrix of type '<class 'numpy.float64'>'
	with 2647 stored elements in Compressed Sparse Row format>

In [33]:
matr_answers_2 = X_start.toarray()

In [34]:
matr_answers_2

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.23484538]])

In [35]:
def text_to_vec_2(text, vectorizer, matr):
    new_doc = vectorizer.transform(text).toarray()
    new_matr = matr.dot(new_doc[0])
    return new_matr

In [36]:
text_2 = text_to_vec_2(queries.head(1)['Текст вопроса'], vectorizer_2, matr_answers_2)

In [37]:
text_2

array([0.05132964, 0.00826135, 0.        , 0.02444088, 0.        ,
       0.        , 0.01655842, 0.02387181, 0.        , 0.05062162,
       0.04710088, 0.18612481, 0.02986892, 0.        , 0.12088349,
       0.        , 0.04099694, 0.        , 0.        , 0.0479808 ,
       0.02159938, 0.06524996, 0.12684607, 0.05579022, 0.        ,
       0.1043222 , 0.0249666 , 0.06173973, 0.05591414, 0.        ,
       0.01440677, 0.12635103, 0.07085367, 0.        , 0.01180225,
       0.11765951, 0.02340343, 0.14332734, 0.16869162, 0.11482521,
       0.02310699, 0.02007746, 0.01955076])

In [38]:
ranging(text_2)

Ранжированные документы по близости: 
---

0.18612481376297696 : номер связки -  308
0.16869161558362822 : номер связки -  135
0.14332733589020907 : номер связки -  6
0.12684606813855775 : номер связки -  286
0.12635102520844527 : номер связки -  270
0.12088349071349433 : номер связки -  1
0.11765950736340813 : номер связки -  88
0.11482520646588391 : номер связки -  5
0.10432220238297966 : номер связки -  37
0.07085366550799661 : номер связки -  246
0.0652499567437455 : номер связки -  325
0.061739732054359675 : номер связки -  74
0.05591413582628506 : номер связки -  132
0.055790216712495955 : номер связки -  79
0.05132963940168474 : номер связки -  57
0.050621616816445836 : номер связки -  89
0.0479807957433737 : номер связки -  34
0.047100880565229285 : номер связки -  173
0.04099693804065354 : номер связки -  10
0.029868921992849802 : номер связки -  46
0.024966597718604842 : номер связки -  32
0.024440881584028892 : номер связки -  327
0.02387181219277522 : номер связки -  43
0.0

### __Задача 2__:    
Аналогичная задаче1 с другой метрикой 

Реализуйте поиск с метрикой *BM25* через умножение матрицы на вектор. Что должно быть в реализации:

- проиндексированная база, где каждый документ представлен в виде вектора BM25
- функция перевода входяшего запроса в вектор по метрике BM25
- ранжирование докуменов по близости к запросу по убыванию

In [39]:
ans = answers['Текст вопросов'].tolist()
n = []
for el in ans:
    c = len(el.split())
    n.append(c)

In [40]:
answers['mead'] = n

In [41]:
df_for_mat = pd.DataFrame(
        matr_answers, columns=vectorizer.get_feature_names())

На текстах после препроцессинга

In [42]:
Q = vectorizer.get_feature_names()

In [43]:
### реализуйте функцию ранжирования 
from math import log

k = 2.0
b = 0.75

N = answers.shape[0]
avgl = answers['mead'].mean()


def bm25(D, el) -> float:
    lngth = len(D.split())
    n_1 = len(df_for_mat[df_for_mat[str(el)] > 0])
    idf = log((N - n_1 + 0.5)/(n_1 + 0.5))
    cnt = 0
    for w in D.split():
        if w == el:
            cnt += 1
    tf = cnt / lngth
    score = idf * ((tf*(k+1))/(tf+k*(1-b+b*(lngth/avgl))))
    return score

In [44]:
all_answ = []
for index, row in answers.iterrows():
    new = []
    for el in Q:
        doc = row['Текст вопросов']
        new.append(bm25(doc, el))
    all_answ.append(new)

In [45]:
all_answ_bm = np.array(all_answ)

In [46]:
all_answ_bm

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.44644048]])

In [47]:
def text_to_vec_bm(text, matr):
    new_doc = []
    for el in Q:
        doc = preprocess_text(text)
        new_doc.append(bm25(doc, el))
    new_matr = matr.dot(new_doc)
    return new_matr

In [48]:
text_bm = text_to_vec_bm(
    queries.head(1)['Текст вопроса'].tolist()[0],
    all_answ_bm)

In [49]:
text_bm

array([0.00374699, 0.        , 0.        , 0.02058998, 0.        ,
       0.        , 0.00304163, 0.00034618, 0.        , 0.00265156,
       0.01410199, 0.00743766, 0.00022265, 0.        , 0.00113193,
       0.        , 0.02870734, 0.        , 0.        , 0.00048392,
       0.00080264, 0.        , 0.00185487, 0.00206367, 0.        ,
       0.00061256, 0.00050266, 0.00351814, 0.00182345, 0.        ,
       0.00014591, 0.00850393, 0.01201601, 0.        , 0.00010563,
       0.05986357, 0.00177231, 0.00103271, 0.00123302, 0.00243822,
       0.00023143, 0.00091524, 0.00421592])

In [50]:
ranging(text_bm)

Ранжированные документы по близости: 
---

0.0598635701949778 : номер связки -  88
0.028707338018106935 : номер связки -  10
0.020589977394909444 : номер связки -  327
0.014101993137105476 : номер связки -  173
0.01201600553731181 : номер связки -  246
0.008503926384007864 : номер связки -  270
0.0074376570140823675 : номер связки -  308
0.004215918820328489 : номер связки -  21
0.0037469864720467384 : номер связки -  57
0.0035181416963293004 : номер связки -  74
0.0030416321244015956 : номер связки -  210
0.0026515603797713644 : номер связки -  89
0.002438224950970566 : номер связки -  5
0.002063665212321624 : номер связки -  79
0.0018548666174708363 : номер связки -  286
0.0018234452323473032 : номер связки -  132
0.0017723052049545232 : номер связки -  70
0.0012330231746194172 : номер связки -  135
0.001131933096176535 : номер связки -  1
0.0010327099757294553 : номер связки -  6
0.0009152365135877355 : номер связки -  45
0.000802635368029204 : номер связки -  38
0.00061255960420763

In [51]:
from tqdm.auto import tqdm

In [52]:
for_bm = []
for index, row in tqdm(queries.iterrows()):
    fin = text_to_vec_bm(row['Текст вопроса'], all_answ_bm)
    for_bm.append(ranging_simple(fin))

In [53]:
queries['bm_vector'] = for_bm

In [56]:
queries.head()

,Текст вопроса,Номер связки,Тематика,Unnamed: 3,Unnamed: 4,tf_vector,bm_vector
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN,37,88
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN,324,328
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN,1,94
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN,32,328
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN,286,328
